In [1]:
# ! git clone https://github.com/roshan-research/hazm.git

In [ ]:
# ! python hazm/setup.py install

In [ ]:
import numpy as np
import nltk
import re
import string
import warnings
import gensim
from sklearn.metrics.pairwise import cosine_similarity
from configparser import ConfigParser
from functools import reduce
from gensim.models import Doc2Vec
from hazm.Embedding import SentEmbedding
from hazm import *

In [ ]:
text = 'پدرم یک کشاورز بودمثل همه که مجبور به این کار بودنالبته شغل‌شو اونطوری شروع نکردسیستم میگه در معرض فشار قرار گرفتینه، مشکلی ندارهاز مخمصه عبور می‌کنیمباید خاموشش کنم، کوپرهمشو خاموش میکنم!نباید چیزی رو خاموش کنیمبابا؟شرمنده، مورف. برگرد بخوابفکر کردم تو روحیگلم روح و اینا وجود ندارهبابابزرگ میگه تو میتونی ارواح رو بگیری...نه، این بخاطر اینکه بابابزرگدیر یا زود خودش از اونا میشه. برو بخوابداشتی خواب سقوط رو میدیدی؟برگرد برو بخواب، مورفمحصولات گندم از بین رفته بودآفت اومده بود و ما مجبور به سوزوندنش بودیمهنوز زمین ذرت داشتیمچندین قطعه مزرعه ذرت داشتیمولی بیشتر وقتا گرد و غبار می‌اومدگمونم نمی‌تونم توضیح بدمطوفان گرد و خاک دائماً می‌اومدهمین، یه عالمه غبار و کثافت...ما خودمونو با پارچه‌هایی می‌پوشوندیمبعضی وقتا روی بینی و دهانمون می‌ذاشتیمکه نتونیم گرد و خاک زیادی رو بدمیمخب زمانی که بشقاب‌ها رو می‌چیدیماونا رو وارونه روی میز می‌ذاشتیملیوان‌ها، بشقاب‌ها، همه چی رو وارونه می‌کردیمبجنبین. مورف، عجله کن...ساعت توی مزرعه، واسه سم ‌پاشیبرای منطقه به روی چشمروی میز نه، مورفبابا، میتونی اینو درست کنی؟چه بلایی سر فضاپیمام آوردی؟من نبودمبذار حدس بزنم. جناب روح این کارو کردبه قفسه کتابم ضربه میزدهمچنین کتاب‌ها رو می‌ندازه زمینکله‌خر، روح وجود خارجی ندارهدربارش تحقیق کردم، بهش میگن شَـبـحبابا، بهش بگوخب، زیادم علمی نیست، مورفخودت گفتی علم درباره پذیرفتنِچیزاییه که ما از اونا نمی‌دونیم!حالا جوابشو بدهمراقب وسایل‌هامون باشکوپخیلی خب، مورف، میخوای علمی حرف بزنی؟،حالا نگو که از روح و اینا می‌ترسینه، تو باید فراتر از این باشی...باید وقایع رو ضبط کنی، آنالیزشون کنیبعد بطور منطقی "چرا و چگونه" این روپیدا کنی و نتایج رو بهم نشون بدی. قبول؟قبولخیلی خبروز خوبی تو مدرسه داشته باشیوایسا،مدرسه جلسه‌ی اولیا تشکیل دادهاولیا، نه اولیای اولیاآروم پسرماون طوفان گرد و خبار نیستنسلون داره محصولاتشو می‌سوزونهآفت؟گفتن که این آخرین برداشتِمحصول واسه بامیه هست.'
keyword_count = 10

In [ ]:
normalizer = Normalizer()
normalize_text = normalizer.normalize(text)
tokenize_text = [word_tokenize(txt) for txt in sent_tokenize(normalize_text)]
tokenize_text

## extract the candidates based on the POS tag of sequence of elements


In [ ]:
grammers = [
"""
NP:
        {<Ne>?<N.*>}    # Noun(s) + Noun(optional) 
        
""",

"""
NP:
        {<N.*><AJ.*>?}    # Noun(s) + Adjective(optional) 
        
"""
]
## you can also add your own grammer to be extracted from the text...

In [ ]:
def extract_candidates(tagged, grammer):
    keyphrase_candidate = set()
    np_parser = nltk.RegexpParser(grammer)
    trees = np_parser.parse_sents(tagged)
    for tree in trees:
        for subtree in tree.subtrees(filter=lambda t: t.label() == 'NP'):  # For each nounphrase
            # Concatenate the token with a space
            keyphrase_candidate.add(' '.join(word for word, tag in subtree.leaves()))
    keyphrase_candidate = {kp for kp in keyphrase_candidate if len(kp.split()) <= 5}
    keyphrase_candidate = list(keyphrase_candidate)
    return keyphrase_candidate    

all_candidates = set()
for grammer in grammers:
    all_candidates.update(extract_candidates(token_tag_list, grammer))

all_candidates = np.array(list(all_candidates))
print(np.array(list(all_candidates)))


## load sent2vec model

In [ ]:
sent2vec_model_path = 'sent2vec-naab.model'
sent2vec_model = SentEmbedding(sent2vec_model_path)

## extract  the embedding vectors from the candidates and the whole text

In [ ]:
all_candidates_vectors = [sent2vec_model[candidate] for candidate in all_candidates]
all_candidates_vectors[0:2]

In [ ]:
whole_text_vector = sent2vec_model[normalize_text]
whole_text_vector.reshape(1,-1)

## find the cosine similarity of the candidates and the whole text

In [ ]:
candidates_sim_whole = cosine_similarity(all_candidates_vectors, whole_text_vector.reshape(1,-1))
candidates_sim_whole.reshape(1,-1)

## the cosine similarity of the each candidates to another

In [ ]:
candidate_sim_candidate = cosine_similarity(all_candidates_vectors)
candidate_sim_candidate

## normalize the value of the cosine similarities

In [ ]:
candidates_sim_whole_norm = candidates_sim_whole / np.max(candidates_sim_whole)
candidates_sim_whole_norm = 0.5 + (candidates_sim_whole_norm - np.average(candidates_sim_whole_norm)) / np.std(candidates_sim_whole_norm)
candidates_sim_whole_norm

In [ ]:
np.fill_diagonal(candidate_sim_candidate, np.NaN)
candidate_sim_candidate_norm = candidate_sim_candidate / np.nanmax(candidate_sim_candidate, axis=0)
candidate_sim_candidate_norm = 0.5 + (candidate_sim_candidate_norm - np.nanmean(candidate_sim_candidate_norm, axis=0)) / np.nanstd(candidate_sim_candidate_norm, axis=0)
candidate_sim_candidate_norm

## extract the proper candidates based on cosine similarity

In [ ]:
beta = 0.82
N = min(len(all_candidates), keyword_count)

selected_candidates = []
unselected_candidates = [i for i in range(len(all_candidates))]
best_candidate = np.argmax(candidates_sim_whole_norm)
selected_candidates.append(best_candidate)
unselected_candidates.remove(best_candidate)


for i in range(N-1):
    selected_vec = np.array(selected_candidates)
    unselected_vec = np.array(unselected_candidates)
    
    unselected_candidate_sim_whole_norm = candidates_sim_whole_norm[unselected_vec, :]
    
    dist_between = candidate_sim_candidate_norm[unselected_vec][:, selected_vec]
    
    if dist_between.ndim == 1:
        dist_between = dist_between[:, np.newaxis]
    
    best_candidate = np.argmax(beta * unselected_candidate_sim_whole_norm - (1 - beta) * np.max(dist_between, axis = 1).reshape(-1,1))
    best_index = unselected_candidates[best_candidate]
    selected_candidates.append(best_index)
    unselected_candidates.remove(best_index)
all_candidates[selected_candidates].tolist()